<a href="https://colab.research.google.com/github/jumbokh/ML-Class/blob/main/notebooks/Ch12HW_%E6%95%B4%E5%90%88%E9%A0%90%E6%B8%AC%E5%99%A8ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 作業
1. 請用鳶尾花全部資料來做分析，並用以下參數進行網格搜尋。

param_grid = [  
    {'model':[RandomForestClassifier()], 'model\_\_n\_estimators': [100, 500]},   
    {'model':[AdaBoostClassifier()], 'model\_\_n\_estimators': [100, 500],  
     'model\_\_base\_estimator':[None, RandomForestClassifier(max_depth=1)]},  
    {'model':[XGBClassifier()], 'model\_\_n\_estimators': [100, 500]},    
]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# plt.rcParams['font.sans-serif'] = ['DFKai-sb'] 
# plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df['target'] = iris['target']
X = df.drop('target', axis=1)
y = df['target']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.33, random_state=42)

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

model_pl = Pipeline([
    ('preprocess', StandardScaler()),
    ('model', LogisticRegression())
])

param_grid = [
    {'model':[RandomForestClassifier()], 
     'model__n_estimators': [100, 500]},
    {'model':[AdaBoostClassifier()], 
     'model__n_estimators': [100, 500],
     'model__base_estimator':[None, RandomForestClassifier(max_depth=1, random_state=42)]},
    {'model':[XGBClassifier()], 
     'model__n_estimators': [100, 500]},
]
gs = GridSearchCV(model_pl, param_grid=param_grid,
                  cv=5, return_train_score=True)
gs.fit(X_train, y_train)
y_pred = gs.best_estimator_.predict(X_test)
score = gs.best_estimator_.score(X_test, y_test)
print('最佳模型', gs.best_estimator_)
print('訓練集交叉驗證的最佳結果', gs.best_score_)
print('測試集的結果', score)
confusion_matrix(y_test, y_pred)

[19:37:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:37:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:37:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:37:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

array([[19,  0,  0],
       [ 0, 15,  0],
       [ 0,  5, 11]])

2. 請用鐵達尼號資料來做分析，並用以下參數進行網格搜尋。

param_grid = [  
    {'model':[RandomForestClassifier()], 'model\_\_n\_estimators': [100, 500]},   
    {'model':[AdaBoostClassifier()], 'model\_\_n\_estimators': [100, 500],  
     'model\_\_base\_estimator':[None, RandomForestClassifier(max_depth=1)]},  
    {'model':[XGBClassifier()], 'model\_\_n\_estimators': [100, 500]},    
]

In [ ]:
df = pd.read_csv('titanic_train.csv')
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.33, random_state=42)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

num_pl = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)
cat_pl = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder()
)

data_pl = ColumnTransformer([
    ('num_pl', num_pl, ['Age','Fare', 'SibSp','Parch','Pclass']),
    ('cat_pl', cat_pl, ['Sex','Embarked'])
])


from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

model_pl = Pipeline([
    ('preprocess', data_pl),
    ('model', LogisticRegression())
])

param_grid = [
    {'model':[RandomForestClassifier()], 
     'model__n_estimators': [100, 500]},
    {'model':[AdaBoostClassifier()], 
     'model__n_estimators': [100, 500],
     'model__base_estimator':[None, RandomForestClassifier(max_depth=1, random_state=42)]},
    {'model':[XGBClassifier()], 
     'model__n_estimators': [100, 500]},
]

gs = GridSearchCV(model_pl, param_grid=param_grid,
                  cv=7, return_train_score=True)
gs.fit(X_train, y_train)
y_pred = gs.best_estimator_.predict(X_test)
score = gs.best_estimator_.score(X_test, y_test)
print('最佳模型', gs.best_params_['model'])
print('訓練集交叉驗證的最佳結果', gs.best_score_)
print('測試集的結果', score)
confusion_matrix(y_test, y_pred)

[19:50:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:50:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:50:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:50:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

array([[144,  31],
       [ 31,  89]])

3. 請用軟投票組合器來進行鐵達尼號存活預測

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

model_pl_lr = make_pipeline(StandardScaler(), LogisticRegression())
model_pl_svc = make_pipeline(StandardScaler(), SVC(probability=True))
model_pl_knn = make_pipeline(StandardScaler(), KNeighborsClassifier())
model_pl_tree = make_pipeline(DecisionTreeClassifier(max_depth=10))

vc = VotingClassifier([
    ('lr', model_pl_lr),    
    ('svc', model_pl_svc), 
    ('tree', model_pl_tree), 
    ('knn', model_pl_knn)], 
    voting='soft', weights=[2, 3, 1, 1])

model_pl = make_pipeline(data_pl, vc)
model_pl.fit(X_train, y_train)
y_pred = model_pl.predict(X_test)

train_score = model_pl.score(X_train, y_train)
test_score = model_pl.score(X_test, y_test)
print('訓練集的預測結果', train_score)
print('測試集的預測結果', test_score)
confusion_matrix(y_test, y_pred)

訓練集的預測結果 0.8557046979865772
測試集的預測結果 0.8271186440677966


array([[161,  14],
       [ 37,  83]])